### Random Forest Assignment_v1.0_01122021

In this assignment students will build the random forest model after
normalizing the variable to house pricing from boston data set.

Task: Deploy this assignment in any cloud platform.(Try to look for free cloud platform)

Assignment: Submit assignment’s deployable link only.

In [1]:
#Following the code to get data into the environment:

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets 
from pandas_profiling import ProfileReport
import pickle
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
#from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import requests




In [3]:

url = "/content/BostonHousing.csv"
boston = pd.read_csv(url)



In [4]:
boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     506 non-null    float64
 1   zn       506 non-null    float64
 2   indus    506 non-null    float64
 3   chas     506 non-null    int64  
 4   nox      506 non-null    float64
 5   rm       506 non-null    float64
 6   age      506 non-null    float64
 7   dis      506 non-null    float64
 8   rad      506 non-null    int64  
 9   tax      506 non-null    int64  
 10  ptratio  506 non-null    float64
 11  b        506 non-null    float64
 12  lstat    506 non-null    float64
 13  medv     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


In [5]:
features = boston.drop(columns=['medv'])

In [6]:
targets = boston.loc[:,['medv']]

In [7]:
features.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [8]:
targets.describe()

,medv
count,506.000000
mean,22.532806
std,9.197104
min,5.000000
25%,17.025000
50%,21.200000
75%,25.000000
max,50.000000


###### CRIM per capita crime rate by town
###### ZN proportion of residential land zoned for lots over 25,000 sq.ft.
###### INDUS proportion of non-retail business acres per town
###### CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
###### NOX nitric oxides concentration (parts per 10 million)
###### RM average number of rooms per dwelling
###### AGE proportion of owner-occupied units built prior to 1940
###### DIS weighted distances to five Boston employment centres
###### RAD index of accessibility to radial highways
###### TAX full-value property-tax rate per 10,000usd
###### PTRATIO pupil-teacher ratio by town
###### B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
###### LSTAT % lower status of the population

In [9]:
features.isnull().sum()

crim       0
zn         0
indus      0
chas       0
nox        0
rm         0
age        0
dis        0
rad        0
tax        0
ptratio    0
b          0
lstat      0
dtype: int64

In [40]:
a = ProfileReport(features)
a.to_notebook_iframe()

TypeError: ignored

In [10]:
X = features
y = targets

In [11]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.30, random_state= 355)

### Random Forests

In [12]:
#let's first visualize the tree on the data without doing any pre processing
clf = DecisionTreeRegressor( min_samples_split= 2)
clf.fit(x_train,y_train)

DecisionTreeRegressor()

In [13]:
# accuracy of our classification tree
clf.score(x_test,y_test)

0.7046285353006798

In [14]:
#let's first visualize the tree on the data without doing any pre processing
clf2 = DecisionTreeRegressor(criterion = 'friedman_mse', max_depth =24, min_samples_leaf= 1)
clf2.fit(x_train,y_train)

DecisionTreeRegressor(criterion='friedman_mse', max_depth=24)

In [15]:
clf2.score(x_test,y_test)

0.826406196046747

In [16]:
rand_clf = RandomForestRegressor(n_estimators=700,random_state=6)

Random state, if given none then score will vary everytime you run the RandomForestClassifier. If we asssign a value to it, then result will remain constant.

In [17]:
rand_clf.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(n_estimators=700, random_state=6)

In [18]:
rand_clf.score(x_test,y_test)

0.8819936389621312

In [19]:
# we are tuning three hyperparameters right now, we are passing the different values for both parameters
grid_param = {
    "n_estimators" : [90,100,115,130],
    'criterion': ['gini', 'entropy'],
    'max_depth' : range(2,20,1),
    'min_samples_leaf' : range(1,10,1),
    'min_samples_split': range(2,10,1),
    'max_features' : ['auto','log2']
}

In [20]:
grid_search = GridSearchCV(estimator=rand_clf,param_grid=grid_param,cv=5,n_jobs =-1,verbose = 3)

In [21]:
grid_search.fit(x_train,y_train)

Fitting 5 folds for each of 20736 candidates, totalling 103680 fits


KeyboardInterrupt: ignored

In [22]:
#let's see the best parameters as per our grid search
grid_search.best_params_

AttributeError: ignored

We will pass these parameters into our random forest classifier.

In [26]:
rand_clf = RandomForestRegressor(criterion= 'mse',
 max_depth = 12,
 max_features = 'log2',
 min_samples_leaf = 1,
 min_samples_split= 5,
 n_estimators = 90,random_state=6)

In [27]:
rand_clf.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:392: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,


RandomForestRegressor(criterion='mse', max_depth=12, max_features='log2',
                      min_samples_split=5, n_estimators=90, random_state=6)

In [28]:
rand_clf.score(x_test,y_test)

0.8810520729644692

In [29]:
rand_clf.predict([[0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98]])[0]

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"


29.192535393618723

In [31]:
# saving the model to the local file system
import pickle
filename = 'boston_randomforest.pickle'
pickle.dump(rand_clf, open(filename, 'wb'))

In [32]:
loaded_model = pickle.load(open(filename, 'rb'))

In [33]:
loaded_model.predict([[0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98]])[0]

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"


29.192535393618723